In [1]:
import glob
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#directory of reference tables 

empCube = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/EmpCube/FACTDATA_MAR201?.TXT'
refData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/REF_Tables/*'

#accensions and seperations source files

accData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/ACC/ACCDATA_FY2011-2017.TXT'
sepData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/SEP/SEPDATA_FY2011-2017.TXT'


def load_emp_fact():

    '''Employment cube load'''

    df_total = pd.DataFrame()

    for name in glob.glob(empCube):


        df = pd.read_csv(name,dtype={ 'AGYSUB': str, 'LOC': str, 'AGELVL': str, 'EDLVL': str, 'GSEGRD': str, 'LOSLVL': str, 'OCC': str, 'PATCO': str,
       'PPGRD': str, 'SALLVL': str, 'STEMOCC': str, 'SUPERVIS': str, 'TOA': str, 'WORKSCH': str, 'WORKSTAT': str,
       'DATECODE': str, 'EMPLOYMENT': str, 'SALARY': str, 'LOS': float})    
        df_total = pd.concat([df_total, df])

    #clean salary col cast to integer
    df_total['SALARY'] = df_total['SALARY'].str.replace('$','')
    df_total['SALARY'] = df_total['SALARY'].str.replace(',','')

    df_total = df_total.dropna(axis = 0, how = 'any')

    df_total['SALARY'] = df_total['SALARY'].astype(int)
    
    df_total.drop(['EDLVL', 'EMPLOYMENT', 'STEMOCC', 'SUPERVIS', 'WORKSTAT'], axis = 1 ,inplace = True)
    
    #just 2018 data 
    df_2018 = df_total.loc[df_total['DATECODE'] == '201803']
    
    df_2018_date = df_2018.drop(['DATECODE'], axis =1)
    
    return df_2018_date


def load_acc_fact():

    '''Load accessions data'''

    print('Loading data...')

    df = pd.read_csv(accData, low_memory=False)

    df = df.dropna(axis = 0, how = 'any')

    df['SALARY'] = df['SALARY'].astype(int)
    
    return df    

def load_sep_fact():

    '''Load separations data'''

    df = pd.read_csv(sepData, low_memory=False)

    df = df.dropna(axis = 0, how = 'any')

    df['SALARY'] = df['SALARY'].astype(int)
    
    df = df[df['OCC'] != '****']

    df = df[df['TOA'] != '**']

    return df




def load_dim_table():

    '''load reference tables'''

    ref_list = []

    for name in glob.glob(refData):

        ref_list.append(pd.read_csv(name))

    return ref_list  



def join_func( df, df_other):

    df = df.merge(df_other)

    return df


def joins(df, data):

    '''indexing based on number and order of reference tables found in the directory '''

    dim = load_dim_table()

    acc = dim[0]
    age = dim[1]
    agency = dim[2]
    education = dim[4]
    date = dim[5]
    grade = dim[6][1:]
    location = dim[7]
    los = dim[8]
    occ = dim[9]
    patco = dim[10]
    paygroup = dim[11]
    salary = dim[12]
    separation = dim[13]
    stem = dim[14]
    supervisor = dim[15]  
    toa = dim[16]
    workstat = dim[17]
    worksch = dim[18]

    # load fact table 

    df_join = df

    '''reference tables joined to acc/sep fact tables'''  

    if (data == 'accessions'):

        df_join = join_func(df_join, acc)
        
        df_join = join_func(df_join, date)
        
        df_join = join_func(df_join, patco)
    
    elif(data =='separations'):

        df_join = join_func(df_join, separation)
        
        df_join = join_func(df_join, date)
        
        df_join = join_func(df_join, patco)
    
    else:
        
        pass

    df_join = join_func(df_join, age)

    df_join = join_func(df_join, agency)    

    df_join = join_func(df_join, location)

    df_join = join_func(df_join, los)

    df_join = join_func(df_join, paygroup)

    df_join = join_func(df_join, salary)

    df_join = join_func(df_join, toa)

    df_join = join_func(df_join, worksch)

    df_join = join_func(df_join, occ)

    return df_join



def join_dim_tables():   

    '''reduce star schema to table'''


    df_joins_acc = joins(load_acc_fact(), 'accessions')

    df_joins_sep = joins(load_sep_fact(), 'separations')


    return df_joins_acc, df_joins_sep  


def raw_data():
    
    acc, sep = join_dim_tables()

    rawData = pd.concat([acc, sep], axis = 0, sort = True)

    rawData['YEAR'] = rawData['EFDATE'].apply(lambda x : x//100)
    
    rawData['QTR'] = rawData['QTR'].astype(str)
    
    # two year data labels and data set

    Year_data = rawData.loc[(rawData['YEAR']>= 2015)]

    X = Year_data.loc[:,('AGELVLT', 'AGYSUBT', 'AGYT', 'AGYTYPT', 'LOCT',
       'LOCTYPT', 'LOS', 'LOSLVLT', 'OCCFAMT', 'PAYPLANT', 'PPGROUPT',
        'PPTYPT', 'SALARY', 'SALLVLT','TOAT', 'TOATYPT', 'WORKSCHT', 'WSTYPT')]


    rawDatalabel = Year_data.loc[:,('ACCT','SEPT')]

    rawDatalabel['ACCT'].update(rawDatalabel.pop('SEPT'))
    
    y_full = rawDatalabel['ACCT'] 
    
    binaryLabel = rawDatalabel['ACCT'].apply(lambda x: 1 if x =='Quit' else 0)

    df_binary = pd.DataFrame(binaryLabel)

    rawDatalabel['ACCT'] = rawDatalabel['ACCT'].apply(lambda x: 'QUIT' if x =='Quit' else 'notQuit')      

    df_label = pd.DataFrame(rawDatalabel['ACCT'] + Year_data['YEAR'].astype(str))

    def labeling(x):
        if x == 'QUIT2015':
            return 't+2'
        elif x == 'QUIT2016':
            return 't+1'
        elif x == 'QUIT2017':
            return 't'
        else:
            return 'Not_Quitting'

    y = df_label[0].apply(labeling)
    
    return X, y, df_binary, y_full
    

def dummies_func(X,y):
    
    X_encoded = pd.get_dummies(X)
    
    y_encoded = pd.get_dummies(y)
    
    return X_encoded, y_encoded

    


def correl(df,features):

    '''Join features with label data and produce correlation matrix'''

    print("Loading correlation matrix...")

    X = df

    df = X[features] 

    df = pd.concat([df,y], axis = 1)  

    df_dummy = pd.get_dummies(df)

    matrix = df_dummy.corr()

    fig, ax = plt.subplots(figsize=(20,20))         

    obj = sns.heatmap(matrix, linewidths=.1, xticklabels=True, yticklabels=True, ax=ax)

    plt.show(obj)

    return obj



def quit_correlation():
    
    a, b, c, d = raw_data()
    
    a_dummy = pd.get_dummies(a)
    
    c = c.rename(columns = {"ACCT": "Quit"})
    
    feature_list = list(a_dummy.columns)

    corr_dict =  {}

    c = c.rename(columns = {"ACCT": "Quit"})

    for feature in feature_list:

        c[feature] = a_dummy[feature]

        matrix = c.corr()

        feature_corr = matrix.iloc[0,1]

        corr_dict.update(({feature:feature_corr}))

        c.drop([feature], axis = 1, inplace = True)
        
    corr_table = pd.DataFrame.from_dict(data = corr_dict, orient = 'index')
    
    corr_table.sort_values(0, ascending = False, inplace = True)
        
    return corr_table

def corr_matrix():
    
    a, b, c, d = raw_data()
    
    rawXy = pd.concat([a, d], axis = 1)
    
    dummyXy = pd.get_dummies(rawXy)
    
    matrix = dummXy.corr()
    
    return matrix   


def model_matrix():
    
    #employment cube 
    
    cube = load_emp_fact()
    
    cube_all = joins(cube, 'none')
    
    cube_red = cube_all[['AGELVLT', 'AGYSUBT', 'AGYT', 'AGYTYPT', 'LOCT', 'LOCTYPT', 'LOS',
       'LOSLVLT', 'OCCFAMT', 'PAYPLANT', 'PPGROUPT', 'PPTYPT', 'SALARY',
       'SALLVLT', 'TOAT', 'TOATYPT', 'WORKSCHT', 'WSTYPT']]
    
    cube_d = pd.get_dummies(cube_red)
    
    #acc/separations confusion matrix 
    
    X, y, c, d = raw_data()

    Xd, yd  = dummies_func(X,y)
    
    # find common features for compatibility 
    
    common = list(set(Xd) & set(cube_d))
    
    Xd_common = Xd[common]
    
    cube_d_common = cube_d[common]   
    
    # confusion matrix 

    cols = yd.columns

    X_train, X_test, y_train, y_test = train_test_split(Xd_common, yd, test_size = .3, random_state = 305)

    model = DecisionTreeClassifier(random_state = 305)

    model.fit(X_train, y_train)

    y_predict_test = model.predict(X_test)

    conf_array = confusion_matrix(y_test.values.argmax(axis =1), y_predict_test.argmax(axis=1), labels = [0, 1, 2, 3])

    conf_df = pd.DataFrame(conf_array, index = cols, columns = cols)
    
    return Xd_common, yd, cube_d_common, model, conf_df


def pred_vols(cube_data):
    
    pred_vols = model.predict(cube_data)
    
    vols = pred_vols.argmax(axis = 1)
    
    df_vols = pd.DataFrame(vols)
    
    df_vols[0].value_counts()
    
    volume_preds = []
    
    for i in range(4):
        
        volume_preds.append(df_vols[0].value_counts()[i])

    volumes = pd.DataFrame(volume_preds, index = ['Not_quiting', '2018', '2019 ', '2020'], columns = ['Projected_quitters'])

    return volumes


def subfieldList(prefix):

    category_list = []

    for col in list(cube_d_common.columns):

        prefix = prefix

        pattern = prefix + '.+'

        startsWith = re.compile(pattern)

        result = startsWith.match(col)

        if result != None:

            category_list.append(result.group(0))

    return category_list  


def subfield_predsdf(subfield):
    
    df = cube_d_common[cube_d_common[subfield] == 1]
    
    return df 

def df_prediction_subfield(prefix, model):
    
    dictionary = {}

    sub_list = subfieldList(prefix)

    for sub in sub_list:

        df_sub = subfield_predsdf(sub)

        predictionsBy = model.predict(df_sub)

        #print(sub)

        #print(predictionsBy.shape)

        df_preds = pd.DataFrame(predictionsBy)

        df_sum = df_preds.sum(axis = 0)

        dictionary.update({sub:[predictionsBy.shape[0] ,df_sum[1], df_sum[2], df_sum[3]]})

    df_out = pd.DataFrame.from_dict(data = dictionary, orient = 'index', columns = ['Current Volume','quit (t)', 'quit (t+1)', 'quit (t+2)'])
    
    return df_out      
    




In [2]:
Xd_common, yd, cube_d_common, model, conf_df = model_matrix()

Loading data...


In [3]:
cube_d_common.shape

(1257656, 841)

In [4]:
prefixList = ['AGELVLT', 'AGYSUBT', 'AGYT', 'AGYTYPT', 'LOCT', 'LOCTYPT', 'LOS',
       'LOSLVLT', 'OCCFAMT', 'PAYPLANT', 'PPGROUPT', 'PPTYPT', 'SALARY',
       'SALLVLT', 'TOAT', 'TOATYPT', 'WORKSCHT', 'WSTYPT']

df = pd.DataFrame([])

for pre in prefixList:
    
    print(pre)
    
    df = df.append(df_prediction_subfield(pre, model))

AGELVLT
AGYSUBT
AGYT
AGYTYPT
LOCT
LOCTYPT
LOS
LOSLVLT
OCCFAMT
PAYPLANT
PPGROUPT
PPTYPT
SALARY
SALLVLT
TOAT
TOATYPT
WORKSCHT
WSTYPT


In [7]:
df

,Current Volume,quit (t),quit (t+1),quit (t+2)
AGELVLT_35-39,155708,19903.0,22712.0,18034.0
AGELVLT_60-64,122592,2756.0,3439.0,2890.0
AGELVLT_50-54,205068,16943.0,19760.0,17383.0
AGELVLT_45-49,172833,17110.0,19820.0,17148.0
AGELVLT_40-44,142216,16801.0,18705.0,16411.0
AGELVLT_55-59,197498,10389.0,12173.0,10355.0
AGELVLT_Less than 20,709,179.0,125.0,77.0
AGELVLT_30-34,120752,15833.0,20230.0,15886.0
AGELVLT_25-29,58497,9254.0,10162.0,8554.0
AGELVLT_20-24,16376,3432.0,3421.0,2088.0


In [6]:
df.to_excel("volumes_by_type_withVols.xlsx")

In [ ]:
df_vol.to_excel("volumes.xlsx")

In [ ]:
heatmap_df.to_excel("heatmap.xlsx")

In [ ]:
df_quit.to_excel("quit_dataframe.xlsx")

In [ ]:
df.to_excel("confusion_matrix.xlsx")